#### WEB專案請求範例JSON資料

In [ ]:
web_json = \
"""
{
	"checkmarx": {
		"enabled": true,
		"version": "latest"
	},
	"newman": {
		"enabled": false
	},
	"web": {
		"type": "php"
	}
}
"""

In [ ]:
import json
import os
from lib_pipeline.rancher_pipeline import env_rancher_pipeline_generate, web_service_pipeline
import yaml
import sys

In [3]:
web_obj = json.loads(web_json)
web_obj

In [4]:
web_pipeline_template = web_service_pipeline(web_obj['checkmarx']['enabled'], web_obj['checkmarx']['version'], web_obj['newman']['enabled'], web_obj['web']['type'])
yaml.dump(web_pipeline_template, sys.stdout)

stages:
- name: Publish User Web image
  steps:
  - env:
      PLUGIN_INSECURE: 'false'
    publishImageConfig:
      buildContext: .
      dockerfilePath: ./Dockerfile
      pushRemote: true
      registry: XX.XX.XX.XX:XXXX
      tag: ${CICD_GIT_REPO_NAME}/${CICD_GIT_BRANCH}-web:latest
- name: Deploy app-web
  steps:
  - applyAppConfig:
      answers:
        checkmarx.enabled: true
        checkmarx.version: latest
        git.branch: ${CICD_GIT_BRANCH}
        git.commitID: ${CICD_GIT_COMMIT}
        git.repoName: ${CICD_GIT_REPO_NAME}
        git.url: ${CICD_GIT_URL}
        newman.enabled: false
        web.image: 10.50.1.63:5443/${CICD_GIT_REPO_NAME}/${CICD_GIT_BRANCH}-web:latest
        web.tag: latest
        web.type: php
      catalogTemplate: cattle-global-data:iii-dev-charts3-web
      name: ${CICD_GIT_REPO_NAME}-${CICD_GIT_BRANCH}-web
      targetNamespace: ${CICD_GIT_REPO_NAME}
      version: 0.1.0
